In [1]:
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)
#若Tensor是通过某些运算得到的，则返回一个与这些运算相关的对象，否则返回None
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [3]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [4]:
# 缺失情况下默认 requires_grad = False
a = torch.randn(2, 2) 
a = ((a * 3) / (a - 1))
print(a)
print(a.requires_grad) # False

a.requires_grad_(True)
print(a.requires_grad) # True

b = (a * a).sum()
print(b.grad_fn)

tensor([[ 1.6903,  0.9353],
        [10.9337,  0.9101]])
False
True


In [5]:
# 等价于 out.backward(torch.tensor(1.))
out.backward() 
# out关于x的梯度
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [6]:
# torch.autograd这个包就是用来计算一些雅克比矩阵的乘积的

In [7]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
# grad在反向传播过程中是累加的(accumulated)
# 每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


In [8]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [9]:
#现在 z 不是一个标量
#在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量。
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


In [10]:
#中断梯度追踪
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

#y2的梯度被torch.no_grad()包裹，与它有关的梯度不会回传
y3.backward()
print(x.grad)

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True
tensor(2.)


In [11]:
x = torch.ones(1,requires_grad=True)

print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播

y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
